https://gist.github.com/kwea123/77ed1640f9bc9550136dc13a6a419e88

In [25]:
from pathlib import Path

import numpy as np
import torch

from models.nerf import Embedding, NeRF
from utils import load_ckpt

import tifffile

In [26]:
chunk = 1024 * 32
img_wh = (3072, 2304)  # full resolution of the input images

# ### Tune these parameters until the whole object lies tightly in range with little noise ###
N = 1024 # controls the resolution, set this number small here because we're only finding
        # good ranges here, not yet for mesh reconstruction; we can set this number high
        # when it comes to final reconstruction.
def get_coords(x):
    cube_size = 4.0
    return x - cube_size / 2, x + cube_size / 2

xmin, xmax = get_coords(0) # left/right range
ymin, ymax = get_coords(0) # forward/backward range
zmin, zmax = get_coords(-4.5) # up/down range
## Attention! the ranges MUST have the same length!
############################################################################################

scene_name = "deer_v6_debug_script_picked_blurry"
EPOCH = 29

In [27]:
EXPERIMENTS = Path("../data/llff/")
CKPTS = Path("ckpts")
RESULTS = Path("results/llff")

In [28]:
root_dir = EXPERIMENTS / f"{scene_name}"  # the folder containing data
root_dir.mkdir(exist_ok=True)

ckpt_path = CKPTS / f"{scene_name}" / f"epoch={EPOCH}.ckpt" # the model path

embedding_xyz = Embedding(3, 10)
embedding_dir = Embedding(3, 4)

nerf_fine = NeRF()
load_ckpt(nerf_fine, ckpt_path, model_name='nerf_fine')
nerf_fine.cuda().eval();

In [29]:
from tqdm import tqdm

x = np.linspace(xmin, xmax, N)
y = np.linspace(ymin, ymax, N)

(RESULTS / f"{scene_name}" / "tiff").mkdir(exist_ok=True) # Create dir if needed
volume_output_file = RESULTS / f"{scene_name}" / "tiff" / f"{scene_name}_volume_N={N}.tiff"
print(str(volume_output_file))
if volume_output_file.exists():
    volume_output_file.unlink()

for z in tqdm(np.linspace(zmin, zmax, N), total=N, leave=False):
    xy_ = torch.FloatTensor(np.stack(np.meshgrid(x, y), -1).reshape(-1, 2)).cuda()
    xyz_ = torch.cat((xy_, torch.full((xy_.shape[0], 1), z).cuda()), dim=1).cuda()

    dir_ = torch.zeros_like(xyz_).cuda()

    with torch.no_grad():
        B = xyz_.shape[0]
        out_chunks = []
        for i in range(0, B, chunk):
            xyz_embedded = embedding_xyz(xyz_[i:i+chunk]) # (N, embed_xyz_channels)
            dir_embedded = embedding_dir(dir_[i:i+chunk]) # (N, embed_dir_channels)
            xyzdir_embedded = torch.cat([xyz_embedded, dir_embedded], 1)
            out_chunks += [nerf_fine(xyzdir_embedded)]
        rgbsigma = torch.cat(out_chunks, 0)

    sigma = rgbsigma[:, -1].cpu().numpy()
    sigma = np.maximum(sigma, 0)
    sigma = sigma.reshape(N, N, 1)

    # save to tiff slice
    exp_path = root_dir
    tifffile.imwrite(volume_output_file, sigma.astype('uint8'), append=True)

results/llff/deer_v6_debug_script_picked_blurry/tiff/deer_v6_debug_script_picked_blurry_volume_N=1024.tiff
